# CanLii API

> Get Metadata for Canadian Law

In [ ]:
#| default_exp canlii_api

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import pandas as pd
import requests
import json
import sys

In [ ]:
#| export
#| hide
class canlii_api:
    "Enter your secret API key and your preferred language ('en' or 'fr')"
    def __init__(self, key='', language='en'):
        self.key = key
        self.language = language
    
    def isolate_id(self, dictionary):
        #used for the list_decisions call
        #some databases return caseId as a language
        try:
            case_id = dictionary[self.language]
        except:
            if self.language == 'en':
                case_id = dictionary['fr']
            else:
                case_id = dictionary['en']
        return case_id
    
    def check_status(self, res):
        if res.status_code == 200:
            pass
        elif res.status_code == 429:
            raise TypeError("Quota exceeded.")
        elif res.status_code == 403:
            raise TypeError("Invalid API key.")
        else:
            raise TypeError(f'Error code {res.status_code}.')
    
        
    def list_tribunals(self):
        "Returns list of CanLii caselaw databases"
        res = requests.get(f'https://api.canlii.org/v1/caseBrowse/{self.language}/?api_key={self.key}')
        self.check_status(res)
        
        json_res = json.loads(res.text)
        df = pd.DataFrame.from_dict(json_res['caseDatabases'])
        return df
    
    def list_decisions(self, databaseId, descending=True, resultCount=10, decision_date_after='', decision_date_before=''):
        if descending == True: 
            offset = 0 
        else: offset = 1

        res = requests.get(f'https://api.canlii.org/v1/caseBrowse/{self.language}/{databaseId}/?offset={offset}&resultCount={resultCount}&decisionDateBefore={decision_date_before}&decisionDateAfter={decision_date_after}&api_key={self.key}')
        self.check_status(res)
            
        json_res = json.loads(res.text)
        df = pd.DataFrame.from_dict(json_res['cases'])
        df['caseId'] = df['caseId'].apply(self.isolate_id)
        df.drop(columns=['databaseId'], inplace=True)
        return df
    
    def case_metadata(self, databaseId, caseId):
        res = requests.get(f'https://api.canlii.org/v1/caseBrowse/{self.language}/{databaseId}/{caseId}/?api_key={self.key}')
        self.check_status(res)
        
        json_res = json.loads(res.text)
        df = pd.Series(json_res)
        return df
    
    def internal_cites(self, databaseId, caseId):
        "Get a list of all the cases your target case cites."
        res = requests.get(f'https://api.canlii.org/v1/caseCitator/en/{databaseId}/{caseId}/citedCases?api_key={self.key}')
        self.check_status(res)

        json_res = json.loads(res.text)
        try:
            df = pd.DataFrame.from_dict(json_res['citedCases'])
            df['caseId'] = df['caseId'].apply(self.isolate_id)
            df.drop(columns=['databaseId'], inplace=True)
        except:
            df = pd.DataFrame()
        return df
    
    def legislation_cites(self, databaseId, caseId):
        "Get a list of all the legislation your target case cites."
        res = requests.get(f'https://api.canlii.org/v1/caseCitator/en/{databaseId}/{caseId}/citedLegislations?api_key={self.key}')
        self.check_status(res)
        
        json_res = json.loads(res.text)
        try:
            df = pd.DataFrame.from_dict(json_res['citedLegislations'])
        except:
            df = pd.DataFrame()
        return df
    
    def external_cites(self, databaseId, caseId):
        "Get a list of all the cases that cite your target case."
        res = requests.get(f'https://api.canlii.org/v1/caseCitator/en/{databaseId}/{caseId}/citingCases?api_key={self.key}')
        self.check_status(res)
        
        json_res = json.loads(res.text)
        try:
            df = pd.DataFrame.from_dict(json_res['citingCases'])
            df['caseId'] = df['caseId'].apply(self.isolate_id)
            df.drop(columns=['databaseId'], inplace=True)
        except:
            df = pd.DataFrame()
        return df
    
    def citations_edge_list(self, df):
    
        citations_edge_list = pd.DataFrame()
        count = 0
        total_number_pulls = df.shape[0]

        for index,row in df.iterrows():
            title = df.loc[index,'title']
            caseId = df.loc[index,'caseId']
            case_citation = df.loc[index,'citation']
            cited_df = api_caller.internal_cites(databaseId=databaseId, caseId=caseId)
            try:
                cited_df['original_caseID'] = caseId
                cited_df['original_case_citation'] = case_citation
                cited_df['original_case_title'] = title

                cited_df.rename(columns={'caseId':'cited_case_caseId', 'citation':'cited_case_citation', 'title':'cited_case_title'}, inplace=True)
                cited_df = cited_df[['original_caseID', 'original_case_citation', 'original_case_title','cited_case_caseId', 'cited_case_title','cited_case_citation']]
            except:
                cited_df = pd.DataFrame()

            citations_edge_list = pd.concat([citations_edge_list,cited_df], axis=0)

            count = count+1
            r = count % 100
            if r == 0:
                print(f'Completed {count} API calls out of {total_number_pulls} requested.')
        
        citations_edge_list.reset_index(drop=True, inplace=True)
        
        return citations_edge_list
    
    def keywords_edge_list(self, df, databaseId=''):
        keywords_edge_list = pd.DataFrame()

        for index,row in df.iterrows():
            databaseId = databaseId
            caseId = df.loc[index,'caseId']
            res = self.case_metadata(databaseId=databaseId, caseId=caseId)

            keywords = res['keywords'].split(' — ')

            case_keywords_df = pd.DataFrame()
            count = 0

            for keyword in keywords:
                case_keywords_df.loc[count,'keyword'] = keyword
                count += 1

            case_keywords_df['databaseId'] = res['databaseId']
            case_keywords_df['caseId'] = res['caseId']
            case_keywords_df['title'] = res['title']
            case_keywords_df['citation'] = res['citation']
            case_keywords_df['decisionDate'] = res['decisionDate']

            case_keywords_df = case_keywords_df[['databaseId', 'caseId', 'title', 'citation', 'decisionDate', 'keyword']]

            keywords_edge_list = pd.concat([keywords_edge_list, case_keywords_df], axis=0)
            
            count = count+1
            r = count % 100
            if r == 0:
                print(f'Completed {count} API calls out of {total_number_pulls} requested.')

        keywords_edge_list.reset_index(drop=True, inplace=True)
        
        return keywords_edge_list
    
    def keywords_edge_list_topic2topic(self, df, databaseId):
        keywords_edge_list = pd.DataFrame()

        for index,row in df.iterrows():
            databaseId = databaseId
            caseId = df.loc[index,'caseId']
            res = self.case_metadata(databaseId=databaseId, caseId=caseId)

            keywords = res['keywords'].split(' — ')

            edges = []

            for keyword in keywords:
                for keyword2 in keywords:
                    if keyword == keyword2:
                        pass
                    else:
                        item = (keyword, keyword2)
                        edges.append(item)
                keywords.remove(keyword)

            case_keywords_df = pd.DataFrame()
            count = 0

            for item in edges:
                case_keywords_df.loc[count,'keyword_1'] = item[0]
                case_keywords_df.loc[count,'keyword_2'] = item[1]
                count += 1

            case_keywords_df['databaseId'] = res['databaseId']
            case_keywords_df['caseId'] = res['caseId']
            case_keywords_df['title'] = res['title']
            case_keywords_df['citation'] = res['citation']
            case_keywords_df['decisionDate'] = res['decisionDate']

            case_keywords_df = case_keywords_df[['databaseId', 'caseId', 'title', 'citation', 'decisionDate', 'keyword_1', 'keyword_2']]

            keywords_edge_list = pd.concat([keywords_edge_list, case_keywords_df], axis=0)
        
        keywords_edge_list.reset_index(drop=True, inplace=True)
        
        return keywords_edge_list

CanLii is an indispensible research tool for lawyers, law students, and scholars. Researchers who wish to interact with CanLii programmatically can access some metadata through an [API](https://github.com/canlii/API_documentation/blob/master/EN.md). This library automates and simplifies many API calls and returns dataframes. Its objective is to help beginners access and use powerful computational tools.

## Program functionality
This module automates calls to CanLii databases to obtain metadata. Out of the box, you can use it to obtain nine different sets of results:

1. Names of all CanLii tribunal databases (database ID, jurisdiction, tribunal name) 
2. Identifying information for decisions in a CanLii database (case ID, title, citation)
3. Metadata for a specific case (information depends on the database)
4. Identities of cases cited by a specified case
5. Identities of statutes cited by a specified case
6. Identities of cases that cite a specified case
7. An edge list of citations (for network analysis)
8. An edge list of keywords (for network analysis)
9. A topic to topic edgelist of keyword (for network analysis)

All results are returned as [Pandas](https://pandas.pydata.org) dataframes or series.

## Demonstration
To get your own API key, request one directly from [CanLii](https://www.canlii.org/en/feedback/feedback.html).

### Initialize the program
```sh
from obiter.canlii_api import *
```

In [ ]:
APIkey = 'YOUR SECRET API KEY'
language = 'en' # or 'fr'

In [ ]:
#| hide
import os

In [ ]:
#| hide
try:
    APIkey = os.environ['CANLII_API_KEY']
except:
    pass

In [ ]:
api_caller = canlii_api(APIkey, language)

### 1. List CanLii caselaw databases

In [ ]:
show_doc(canlii_api.list_tribunals, title_level=4)

---

#### canlii_api.list_tribunals

>      canlii_api.list_tribunals ()

Returns list of CanLii caselaw databases

In [ ]:
api_caller.list_tribunals()

,databaseId,jurisdiction,name
0,qccdoooq,qc,Conseil de discipline de l'Ordre des opticiens...
1,skqb,sk,Court of King's Bench for Saskatchewan
2,qcoaciq,qc,Comité de discipline de l'organisme d'autorégl...
3,abmgb,ab,Alberta Municipal Government Board
4,onsc,on,Superior Court of Justice
...,...,...,...
357,ytrto,yk,Yukon Residential Tenancies Office
358,nbsec,nb,Financial and Consumer Services Tribunal
359,onbcc,on,Building Code Commission
360,exchc-cech,ca,Exchequer Court of Canada


In [ ]:
#isolate a tribunal
isolate_df = api_caller.list_tribunals()

In [ ]:
filt = isolate_df['name'].str.contains('Federal')
isolate_df[filt]

,databaseId,jurisdiction,name
128,fca,ca,Federal Court of Appeal
246,fct,ca,Federal Court
247,pslreb,ca,Federal Public Sector Labour Relations and Emp...
264,caci,ca,Federal Commission of Inquiry


### 2. List decisions in a caselaw database

In [ ]:
show_doc(canlii_api.list_decisions, title_level=4)

---

#### canlii_api.list_decisions

>      canlii_api.list_decisions (databaseId, descending=True, resultCount=10,
>                                 decision_date_after='',
>                                 decision_date_before='')

In [ ]:
databaseId = 'onbcc' #ID of database to be searched, (Building Code Commission)
resultCount = 10 #number of results
decision_date_after = '2020-01-01' # "date" is a YYYY-MM-DD
decision_date_before = '2020-12-31'

api_caller.list_decisions(databaseId=databaseId, descending=False, resultCount=resultCount, decision_date_after=decision_date_after, decision_date_before=decision_date_before)

,caseId,title,citation
0,2020onbcc19,Leung v. Johnston,2020 ONBCC 19 (CanLII)
1,2020onbcc20,Earle v. Richardson,2020 ONBCC 20 (CanLII)
2,2020onbcc15,Oster v. Bidin,2020 ONBCC 15 (CanLII)
3,2020onbcc14,Ropp v. Capener-Hunt,2020 ONBCC 14 (CanLII)
4,2020onbcc11,Vance v. Cox,2020 ONBCC 11 (CanLII)
5,2020onbcc8,MacPhail v. VanderWindt,2020 ONBCC 8 (CanLII)
6,2020onbcc6,Randle v. Johnston,2020 ONBCC 6 (CanLII)
7,2020onbcc3,Hand v. Janotta,2020 ONBCC 3 (CanLII)
8,2020onbcc2,Hooshley v. Goodale,2020 ONBCC 2 (CanLII)
9,2020onbcc1,Allen v. Fedoriw,2020 ONBCC 1 (CanLII)


### 3. Get case metadata 

In [ ]:
show_doc(canlii_api.case_metadata, title_level=4)

---

#### canlii_api.case_metadata

>      canlii_api.case_metadata (databaseId, caseId)

In [ ]:
databaseId = 'onbcc' #ID of database to be searched, (Building Code Commission)
caseId = '2021onbcc13' #Id of target case

api_caller.case_metadata(databaseId=databaseId, caseId=caseId)

databaseId                                                    onbcc
caseId                                                  2021onbcc13
url                                       https://canlii.ca/t/jkw96
title                                 Johnston and Roach v. Findlay
citation                                     2021 ONBCC 13 (CanLII)
language                                                         en
docketNumber                                  B 2021-06; 21-13-1187
decisionDate                                             2021-07-15
keywords          sound attenuation — second dwelling unit — chi...
topics                                                             
concatenatedId                                          2021onbcc13
dtype: object

### 4. Internal citations (i.e. cases a target case cites)

In [ ]:
show_doc(canlii_api.internal_cites, title_level=4)

---

#### canlii_api.internal_cites

>      canlii_api.internal_cites (databaseId, caseId)

Get a list of all the cases your target case cites.

In [ ]:
databaseId = 'skqb' #ID of database to be searched, (Court of King's Bench for Saskatchewan)
caseId = '2022skkb215' #Id of target case

api_caller.internal_cites(databaseId=databaseId, caseId=caseId)

,caseId,title,citation
0,1974canlii14,Kienapple v. R.,"1974 CanLII 14 (SCC), [1975] 1 SCR 729"
1,2022canlii26225,Michael Louis Hamer v. Her Majesty the Queen,2022 CanLII 26225 (SCC)
2,2022onca553,R. v. A.R.,2022 ONCA 553 (CanLII)
3,2007onca421,R. v. Allen,2007 ONCA 421 (CanLII)
4,2017scc64,R. v. Boutilier,"2017 SCC 64 (CanLII), [2017] 2 SCR 936"
5,1997canlii347,R. v. Currie,"1997 CanLII 347 (SCC), [1997] 2 SCR 260"
6,2020scc9,R. v. Friesen,2020 SCC 9 (CanLII)
7,1999canlii679,R. v. Gladue,"1999 CanLII 679 (SCC), [1999] 1 SCR 688"
8,2021bcca297,R. v. Hamer,2021 BCCA 297 (CanLII)
9,2011onca840,R. v. Hogg,2011 ONCA 840 (CanLII)


### 5. External citations (i.e. cases that cite a target case)

In [ ]:
show_doc(canlii_api.external_cites, title_level=4)

---

#### canlii_api.external_cites

>      canlii_api.external_cites (databaseId, caseId)

Get a list of all the cases that cite your target case.

In [ ]:
databaseId = 'csc-scc' #ID of database to be searched, (Supreme Court of Canada)
caseId = '2022scc30' #Id of target case

api_caller.external_cites(databaseId=databaseId, caseId=caseId)

,caseId,title,citation
0,2022cb7,SOCAN Tariff 22.G – Game Sites (2007-2019),2022 CB 7 (CanLII)
1,2022qccs3010,Fédération des travailleurs et travailleuses d...,2022 QCCS 3010 (CanLII)
2,2022qcca1139,Ville de Brossard c. Ville de Longueuil,2022 QCCA 1139 (CanLII)
3,2022abqb611,NAIT Students’ Association v NAIT,2022 ABKB 611 (CanLII)
4,2022fc1147,Makkar v. Canada (Citizenship and Immigration),2022 FC 1147 (CanLII)
5,2022qccs3728,Hydro-Québec c. Régie de l'Énergie,2022 QCCS 3728 (CanLII)
6,2022fc1365,Afzal v. Canada (Citizenship and Immigration),2022 FC 1365 (CanLII)
7,2022abca264,Sedgwick v Edmonton Real Estate Board Co-Opera...,2022 ABCA 264 (CanLII)
8,2022abca259,Fazel v Singer (Wilson Laycraft),2022 ABCA 259 (CanLII)
9,2022qccs3465,Ville de Gatineau c. Association des pompiers ...,2022 QCCS 3465 (CanLII)


### 6. Legislation citations within a case

In [ ]:
show_doc(canlii_api.legislation_cites, title_level=4)

---

#### canlii_api.legislation_cites

>      canlii_api.legislation_cites (databaseId, caseId)

Get a list of all the legislation your target case cites.

In [ ]:
databaseId = 'csc-scc' #ID of database to be searched, (Supreme Court of Canada)
caseId = '2022scc30' #Id of target case

api_caller.legislation_cites(databaseId=databaseId, caseId=caseId)

,databaseId,legislationId,title,citation,type
0,cas,rsc-1985-c-c-42,Copyright Act,"RSC 1985, c C-42",STATUTE
1,caa,sc-2012-c-20,Copyright Modernization Act,"SC 2012, c 20",ANNUAL_STATUTE


### 7. Citations for network analyses (edge list)

In [ ]:
show_doc(canlii_api.citations_edge_list, title_level=4)

---

#### canlii_api.citations_edge_list

>      canlii_api.citations_edge_list (df)

In [ ]:
databaseId = 'csc-scc' #ID of database to be searched, (Building Code Commission)
resultCount = 3000 #number of results
decision_date_after = '2021-01-01' # "date" is a YYYY-MM-DD
decision_date_before = '2021-12-31'

df = api_caller.list_decisions(databaseId=databaseId, descending=False, resultCount=resultCount, decision_date_after=decision_date_after, decision_date_before=decision_date_before)

In [ ]:
api_caller.citations_edge_list(df)

,original_caseID,original_case_citation,original_case_title,cited_case_caseId,cited_case_title,cited_case_citation
0,2021scc53,2021 SCC 53 (CanLII),Montréal (City) v. Deloitte Restructuring Inc.,2020scc10,9354-9186 Québec inc. v. Callidus Capital Corp.,2020 SCC 10 (CanLII)
1,2021scc53,2021 SCC 53 (CanLII),Montréal (City) v. Deloitte Restructuring Inc.,2009abqb183,Agriculture Financial Services Corporation v. ...,2009 ABQB 183 (CanLII)
2,2021scc53,2021 SCC 53 (CanLII),Montréal (City) v. Deloitte Restructuring Inc.,2003canlii64234,Air Canada (Re),2003 CanLII 64234 (ON SC)
3,2021scc53,2021 SCC 53 (CanLII),Montréal (City) v. Deloitte Restructuring Inc.,2020qcca438,Arrangement relatif à Consultants SM inc.,2020 QCCA 438 (CanLII)
4,2021scc53,2021 SCC 53 (CanLII),Montréal (City) v. Deloitte Restructuring Inc.,2019qccs2316,Arrangement relatif à Consultants SM inc.,2019 QCCS 2316 (CanLII)
...,...,...,...,...,...,...
1865,2021scc2,2021 SCC 2 (CanLII),R. v. Yusuf,2017scc31,R. v. Cody,"2017 SCC 31 (CanLII), [2017] 1 SCR 659"
1866,2021scc2,2021 SCC 2 (CanLII),R. v. Yusuf,2016scc27,R. v. Jordan,"2016 SCC 27 (CanLII), [2016] 1 SCR 631"
1867,2021scc2,2021 SCC 2 (CanLII),R. v. Yusuf,2020onca220,R. v. Pauls,2020 ONCA 220 (CanLII)
1868,2021scc1,2021 SCC 1 (CanLII),Armstrong v. Ward,2019onca963,Armstrong v. Royal Victoria Hospital,2019 ONCA 963 (CanLII)


### 8. Keywords for network analyses (edge list)

In [ ]:
show_doc(canlii_api.keywords_edge_list, title_level=4)

---

#### canlii_api.keywords_edge_list

>      canlii_api.keywords_edge_list (df, databaseId='')

In [ ]:
databaseId = 'csc-scc' #ID of database to be searched, (Supreme Court of Canada)
resultCount = 200 #max number of results
decision_date_after = '2021-01-01' # "date" is a YYYY-MM-DD
decision_date_before = '2021-12-31'

df = api_caller.list_decisions(databaseId=databaseId, descending=False, resultCount=resultCount, decision_date_after=decision_date_after, decision_date_before=decision_date_before)

In [ ]:
df

,caseId,title,citation
0,2021scc53,Montréal (City) v. Deloitte Restructuring Inc.,2021 SCC 53 (CanLII)
1,2021scc52,R. v. Lai,2021 SCC 52 (CanLII)
2,2021scc51,Canada v. Loblaw Financial Holdings Inc.,2021 SCC 51 (CanLII)
3,2021scc50,Kreke v. Alansari,2021 SCC 50 (CanLII)
4,2021scc49,Canada v. Alta Energy Luxembourg S.A.R.L.,2021 SCC 49 (CanLII)
5,2021scc48,R. v. Albashir,2021 SCC 48 (CanLII)
6,2021scc47,Trial Lawyers Association of British Columbia ...,2021 SCC 47 (CanLII)
7,2021scc46,R. v. Parranto,2021 SCC 46 (CanLII)
8,2021scc45,R. v. Cowan,2021 SCC 45 (CanLII)
9,2021scc44,H.M.B. Holdings Ltd. v. Antigua and Barbuda,2021 SCC 44 (CanLII)


In [ ]:
api_caller.keywords_edge_list(df, databaseId=databaseId)

,databaseId,caseId,title,citation,decisionDate,keyword
0,csc-scc,2021scc53,Montréal (City) v. Deloitte Restructuring Inc.,2021 SCC 53 (CanLII),2021-12-10,pre-post compensation
1,csc-scc,2021scc53,Montréal (City) v. Deloitte Restructuring Inc.,2021 SCC 53 (CanLII),2021-12-10,pre-post set-off
2,csc-scc,2021scc53,Montréal (City) v. Deloitte Restructuring Inc.,2021 SCC 53 (CanLII),2021-12-10,creditor
3,csc-scc,2021scc53,Montréal (City) v. Deloitte Restructuring Inc.,2021 SCC 53 (CanLII),2021-12-10,debtor
4,csc-scc,2021scc53,Montréal (City) v. Deloitte Restructuring Inc.,2021 SCC 53 (CanLII),2021-12-10,supervising
...,...,...,...,...,...,...
260,csc-scc,2021scc1,Armstrong v. Ward,2021 SCC 1 (CanLII),2021-01-18,ureter
261,csc-scc,2021scc1,Armstrong v. Ward,2021 SCC 1 (CanLII),2021-01-18,intervener
262,csc-scc,2021scc1,Armstrong v. Ward,2021 SCC 1 (CanLII),2021-01-18,kidney
263,csc-scc,2021scc1,Armstrong v. Ward,2021 SCC 1 (CanLII),2021-01-18,colectomy


### 9. Topic to Topic keyword edge list

In [ ]:
show_doc(canlii_api.keywords_edge_list_topic2topic, title_level=4)

---

#### canlii_api.keywords_edge_list_topic2topic

>      canlii_api.keywords_edge_list_topic2topic (df, databaseId)

In [ ]:
databaseId = 'csc-scc' #ID of database to be searched, (Supreme Court of Canada)
resultCount = 200 #mac number of results
decision_date_after = '2021-01-01' # "date" is a YYYY-MM-DD
decision_date_before = '2021-12-31'

df = api_caller.list_decisions(databaseId=databaseId, descending=False, resultCount=resultCount, decision_date_after=decision_date_after, decision_date_before=decision_date_before)

In [ ]:
api_caller.keywords_edge_list_topic2topic(df, databaseId=databaseId)

,databaseId,caseId,title,citation,decisionDate,keyword_1,keyword_2
0,csc-scc,2021scc53,Montréal (City) v. Deloitte Restructuring Inc.,2021 SCC 53 (CanLII),2021-12-10,pre-post compensation,pre-post set-off
1,csc-scc,2021scc53,Montréal (City) v. Deloitte Restructuring Inc.,2021 SCC 53 (CanLII),2021-12-10,pre-post compensation,creditor
2,csc-scc,2021scc53,Montréal (City) v. Deloitte Restructuring Inc.,2021 SCC 53 (CanLII),2021-12-10,pre-post compensation,debtor
3,csc-scc,2021scc53,Montréal (City) v. Deloitte Restructuring Inc.,2021 SCC 53 (CanLII),2021-12-10,pre-post compensation,supervising
4,csc-scc,2021scc53,Montréal (City) v. Deloitte Restructuring Inc.,2021 SCC 53 (CanLII),2021-12-10,creditor,pre-post set-off
...,...,...,...,...,...,...,...
472,csc-scc,2021scc1,Armstrong v. Ward,2021 SCC 1 (CanLII),2021-01-18,kidney,intervener
473,csc-scc,2021scc1,Armstrong v. Ward,2021 SCC 1 (CanLII),2021-01-18,kidney,colectomy
474,csc-scc,2021scc1,Armstrong v. Ward,2021 SCC 1 (CanLII),2021-01-18,kidney,laparoscopic
475,csc-scc,2021scc1,Armstrong v. Ward,2021 SCC 1 (CanLII),2021-01-18,laparoscopic,intervener


## Saving and manipulating data
All program outputs are Pandas dataframes. To learn more about Pandas, check out the library's [introductory guide](https://pandas.pydata.org/docs/getting_started/index.html).

Basic functionality:

In [ ]:
# save a datafram as a CSV file (to run in Excel, etc.)
df.to_csv('filename.csv')

# filter results
filt = df['title'].str.contains('R. v.')
df[filt]

,caseId,title,citation
1,2021scc52,R. v. Lai,2021 SCC 52 (CanLII)
5,2021scc48,R. v. Albashir,2021 SCC 48 (CanLII)
7,2021scc46,R. v. Parranto,2021 SCC 46 (CanLII)
8,2021scc45,R. v. Cowan,2021 SCC 45 (CanLII)
13,2021scc40,R. v. Strathdee,2021 SCC 40 (CanLII)
15,2021scc38,R. v. Reilly,2021 SCC 38 (CanLII)
16,2021scc37,R. v. Khill,2021 SCC 37 (CanLII)
18,2021scc35,R. v. Dingwall,2021 SCC 35 (CanLII)
27,2021scc26,R. v. Chouhan,2021 SCC 26 (CanLII)
32,2021scc21,R. v. Morrow,2021 SCC 21 (CanLII)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()